In [122]:
from attributes import *
import polars as pl
import altair as alt

# get data
df = pl.read_csv("data/2_research_ethic_recognition.csv")

In [123]:
def basic_analysis(name):
    count = df.group_by(name).len().sort(name)
    proportion = count.with_columns(
        pct = pl.col("len") / pl.col("len").sum() * 100
    )
    mean = df.select(mean = pl.col(name).mean())
    std = df.select(std = pl.col(name).std())
    

In [124]:
def draw_bar_plot(df, x, y, title, MAP, rotate=True, legend=True):

    bar_x = alt.X(x, title=title, axis=None)
    bar_x = bar_x if MAP is None else bar_x.sort(MAP.values())

    txt_x = alt.X(x, sort=MAP.values()) if MAP is not None else alt.X(x)
    lgd_x = alt.Legend(orient='bottom', direction='horizontal')
    color = alt.Color(x, title=None, legend=lgd_x) if legend else alt.Color(x, title=None, legend=None)
    color = color.sort(MAP.values()) if MAP is not None else color

    bar = df.plot.bar().encode(
        x=bar_x,
        y=alt.Y(y, title=None),
        color=color,
    ).properties(
        title=title,
        width=400,
    )

    text = bar.mark_text(
        align="center",
        baseline="bottom",
    ).encode(
        x=txt_x,
        y=y,
        text=y,
        color=alt.value("black"),
    )
    return bar + text

In [125]:
def get(title, __2022, __2023, importance, name="응답"):

    __2022 = __2022.with_columns(
        pct2022 = pl.col(title) / pl.col(title).sum() * 100,
    )

    __2023 = __2023.with_columns(
        pct2023 = pl.col(title) / pl.col(title).sum() * 100,
    )

    _2024 = df.group_by(title).len().sort(title)

    if _2024[title].len() < len(importance.keys()):
        extending = pl.DataFrame({
            title: [1],
            "len": [0]
        }).cast(pl.Schema({ "len": pl.UInt32()}))
        _2024 = _2024.extend(extending)

    _2024 = _2024.rename({"len": "2024"}).sort(title)
    _2024 = _2024.with_columns(
        pct2024 = pl.col("2024") / pl.col("2024").sum() * 100,
        mean2024 = (pl.col(title) * pl.col("2024")).sum() / _2024["2024"].sum(),
    )
    _2024 = _2024.with_columns(
        std2024 = (((pl.col(title) - _2024["mean2024"]) ** 2 * pl.col("2024")).sum() / pl.col("2024").sum()).sqrt(),
    )

    all = __2022.join(__2023, on=title, how="left").join(_2024, on=title, how="left")
    all = all.with_columns(
        total = pl.col("2022") + pl.col("2023") + pl.col("2024")
    )

    all = all.with_columns(
        pctAll = all["total"] / all["total"].sum() * 100,
        meanAll = (pl.col(title) * pl.col("total")).sum() / all["total"].sum(),
    )
    all = all.with_columns(
        stdAll = (((pl.col(title) - all["meanAll"]) ** 2 * pl.col("total")).sum() / pl.col("total").sum()).sqrt(),
    )

    name = pl.DataFrame({name: importance.values()})
    all = all.with_columns(
        name
    )
    
    print(all)

    return all

In [126]:
def draw_graph(_data, title, sorted):
    data_long = _data.select("응답", "2022", "2023", "2024").melt(id_vars=["응답"], variable_name="Year", value_name="Count")

    stacked_bar = alt.Chart(data_long).mark_bar(size=30).encode(
        x=alt.X("Year:O", title=None, axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("Count:Q", title=None).stack('normalize'),
        color=alt.Color(f"응답:N", title=None).scale(scheme="category20").sort(sorted.values()),
        order=alt.Order('Count', sort='descending')
        # tooltip=[title[0], "Year", "Count"]
    ).configure_axis(
        labelFontSize=15,
        titleFontSize=15,
        titlePadding=100
    ).properties(
        width=420,
        height=296,
        title=title,
        padding={'top':20, "bottom": 10, "left": 10, "right": 10}
    ).configure_title(
        subtitlePadding=100,
        offset=10,        
    )

    stacked_bar.save(f"figure/7/{title}.png")
    stacked_bar.show()

In [127]:
# 
# 가. 연구자의 연구윤리 인식과 실천의 중요성 인식 
# 
title = "연구윤리 인식 수준 평균 변화"

yesorno_5 = {
    1: "전혀 그렇지 않다", 
    2: "그렇지 않다",
    3: "보통이다", 
    4: "그렇다", 
    5: "매우 그렇다",
}

Q = "Q1"

# 1) counting
_2022 = pl.DataFrame({
    "Q1": yesorno_5.keys(),
    "2022": [9, 0, 4, 47, 103],
})

_2023 = pl.DataFrame({
    "Q1": yesorno_5.keys(),
    "2023": [0, 0, 2, 56, 112],
})

# basic_analysis("Q1")
data = get(Q, _2022, _2023, yesorno_5)
draw_graph(data, title, yesorno_5)

shape: (5, 14)
┌─────┬──────┬───────────┬──────┬───┬────────┬─────────┬──────────┬──────────────────┐
│ Q1  ┆ 2022 ┆ pct2022   ┆ 2023 ┆ … ┆ pctAll ┆ meanAll ┆ stdAll   ┆ 응답             │
│ --- ┆ ---  ┆ ---       ┆ ---  ┆   ┆ ---    ┆ ---     ┆ ---      ┆ ---              │
│ i64 ┆ i64  ┆ f64       ┆ i64  ┆   ┆ f64    ┆ f64     ┆ f64      ┆ str              │
╞═════╪══════╪═══════════╪══════╪═══╪════════╪═════════╪══════════╪══════════════════╡
│ 1   ┆ 9    ┆ 6.666667  ┆ 0    ┆ … ┆ 1.8    ┆ 4.58    ┆ 0.709648 ┆ 전혀 그렇지 않다 │
│ 2   ┆ 0    ┆ 13.333333 ┆ 0    ┆ … ┆ 0.2    ┆ 4.58    ┆ 0.709648 ┆ 그렇지 않다      │
│ 3   ┆ 4    ┆ 20.0      ┆ 2    ┆ … ┆ 1.6    ┆ 4.58    ┆ 0.709648 ┆ 보통이다         │
│ 4   ┆ 47   ┆ 26.666667 ┆ 56   ┆ … ┆ 31.0   ┆ 4.58    ┆ 0.709648 ┆ 그렇다           │
│ 5   ┆ 103  ┆ 33.333333 ┆ 112  ┆ … ┆ 65.4   ┆ 4.58    ┆ 0.709648 ┆ 매우 그렇다      │
└─────┴──────┴───────────┴──────┴───┴────────┴─────────┴──────────┴──────────────────┘


C:\Users\hyeon\AppData\Local\Temp\ipykernel_13024\3325326262.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select("응답", "2022", "2023", "2024").melt(id_vars=["응답"], variable_name="Year", value_name="Count")


alt.Chart(...)

In [128]:
# from scipy import stats
# # 
# # 나. 연구자의 연구윤리 준수 수준 
# # 

# # # 1) counting
# # _2022 = pl.DataFrame({
# #     "Q2": importance,
# #     "2022": [1, 5, 42, 89, 26],
# # })

# # _2023 = pl.DataFrame({
# #     "Q2": importance,
# #     "2023": [0, 6, 55, 75, 34],
# # })

# # get("Q2", _2022, _2023)

In [129]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
title = "기관 내 연구윤리 교육 중요도"

importance = {
    1: "전혀 중요하지 않다", 
    2: "중요하지 않다",
    3: "보통이다", 
    4: "중요하다", 
    5: "매우 중요하다",
}

Q ="Q3_1"

# 1) counting
_2022 = pl.DataFrame({
    Q: importance.keys(),
    "2022": [1, 2, 33, 88, 39],
})

_2023 = pl.DataFrame({
    Q: importance.keys(),
    "2023": [2, 4, 38, 90, 36],
})

data = get(Q, _2022, _2023, importance)
draw_graph(data, title, importance)

shape: (5, 14)
┌──────┬──────┬───────────┬──────┬───┬────────┬─────────┬──────────┬────────────────────┐
│ Q3_1 ┆ 2022 ┆ pct2022   ┆ 2023 ┆ … ┆ pctAll ┆ meanAll ┆ stdAll   ┆ 응답               │
│ ---  ┆ ---  ┆ ---       ┆ ---  ┆   ┆ ---    ┆ ---     ┆ ---      ┆ ---                │
│ i64  ┆ i64  ┆ f64       ┆ i64  ┆   ┆ f64    ┆ f64     ┆ f64      ┆ str                │
╞══════╪══════╪═══════════╪══════╪═══╪════════╪═════════╪══════════╪════════════════════╡
│ 1    ┆ 1    ┆ 6.666667  ┆ 2    ┆ … ┆ 1.0    ┆ 3.996   ┆ 0.779733 ┆ 전혀 중요하지 않다 │
│ 2    ┆ 2    ┆ 13.333333 ┆ 4    ┆ … ┆ 2.0    ┆ 3.996   ┆ 0.779733 ┆ 중요하지 않다      │
│ 3    ┆ 33   ┆ 20.0      ┆ 38   ┆ … ┆ 18.6   ┆ 3.996   ┆ 0.779733 ┆ 보통이다           │
│ 4    ┆ 88   ┆ 26.666667 ┆ 90   ┆ … ┆ 53.2   ┆ 3.996   ┆ 0.779733 ┆ 중요하다           │
│ 5    ┆ 39   ┆ 33.333333 ┆ 36   ┆ … ┆ 25.2   ┆ 3.996   ┆ 0.779733 ┆ 매우 중요하다      │
└──────┴──────┴───────────┴──────┴───┴────────┴─────────┴──────────┴────────────────────┘


C:\Users\hyeon\AppData\Local\Temp\ipykernel_13024\3325326262.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select("응답", "2022", "2023", "2024").melt(id_vars=["응답"], variable_name="Year", value_name="Count")


alt.Chart(...)

In [130]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
title = "연구윤리 관련 보도의 중요도"
Q ="Q3_2"

# 1) counting
_2022 = pl.DataFrame({
    Q: importance.keys(),
    "2022": [1, 5, 42, 89, 26],
})

_2023 = pl.DataFrame({
    Q: importance.keys(),
    "2023": [0, 6, 55, 75, 34],
})

data = get(Q, _2022, _2023, importance)
draw_graph(data, title, importance)

shape: (5, 14)
┌──────┬──────┬───────────┬──────┬───┬────────┬─────────┬──────────┬────────────────────┐
│ Q3_2 ┆ 2022 ┆ pct2022   ┆ 2023 ┆ … ┆ pctAll ┆ meanAll ┆ stdAll   ┆ 응답               │
│ ---  ┆ ---  ┆ ---       ┆ ---  ┆   ┆ ---    ┆ ---     ┆ ---      ┆ ---                │
│ i64  ┆ i64  ┆ f64       ┆ i64  ┆   ┆ f64    ┆ f64     ┆ f64      ┆ str                │
╞══════╪══════╪═══════════╪══════╪═══╪════════╪═════════╪══════════╪════════════════════╡
│ 1    ┆ 1    ┆ 6.666667  ┆ 0    ┆ … ┆ 1.2    ┆ 3.832   ┆ 0.814724 ┆ 전혀 중요하지 않다 │
│ 2    ┆ 5    ┆ 13.333333 ┆ 6    ┆ … ┆ 3.2    ┆ 3.832   ┆ 0.814724 ┆ 중요하지 않다      │
│ 3    ┆ 42   ┆ 20.0      ┆ 55   ┆ … ┆ 26.2   ┆ 3.832   ┆ 0.814724 ┆ 보통이다           │
│ 4    ┆ 89   ┆ 26.666667 ┆ 75   ┆ … ┆ 50.0   ┆ 3.832   ┆ 0.814724 ┆ 중요하다           │
│ 5    ┆ 26   ┆ 33.333333 ┆ 34   ┆ … ┆ 19.4   ┆ 3.832   ┆ 0.814724 ┆ 매우 중요하다      │
└──────┴──────┴───────────┴──────┴───┴────────┴─────────┴──────────┴────────────────────┘


C:\Users\hyeon\AppData\Local\Temp\ipykernel_13024\3325326262.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select("응답", "2022", "2023", "2024").melt(id_vars=["응답"], variable_name="Year", value_name="Count")


alt.Chart(...)

In [131]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
title = "동료‧연구자 간 대화 및 멘토링의 중요도"
Q ="Q3_3"

# 1) counting
_2022 = pl.DataFrame({
    Q: importance.keys(),
    "2022": [1, 1, 13, 81, 67],
})

_2023 = pl.DataFrame({
    Q: importance.keys(),
    "2023": [0, 3, 18, 8, 65],
})

data = get(Q, _2022, _2023, importance)
draw_graph(data, title, importance)

shape: (5, 14)
┌──────┬──────┬───────────┬──────┬───┬───────────┬──────────┬──────────┬────────────────────┐
│ Q3_3 ┆ 2022 ┆ pct2022   ┆ 2023 ┆ … ┆ pctAll    ┆ meanAll  ┆ stdAll   ┆ 응답               │
│ ---  ┆ ---  ┆ ---       ┆ ---  ┆   ┆ ---       ┆ ---      ┆ ---      ┆ ---                │
│ i64  ┆ i64  ┆ f64       ┆ i64  ┆   ┆ f64       ┆ f64      ┆ f64      ┆ str                │
╞══════╪══════╪═══════════╪══════╪═══╪═══════════╪══════════╪══════════╪════════════════════╡
│ 1    ┆ 1    ┆ 6.666667  ┆ 0    ┆ … ┆ 0.235849  ┆ 4.360849 ┆ 0.736178 ┆ 전혀 중요하지 않다 │
│ 2    ┆ 1    ┆ 13.333333 ┆ 3    ┆ … ┆ 1.415094  ┆ 4.360849 ┆ 0.736178 ┆ 중요하지 않다      │
│ 3    ┆ 13   ┆ 20.0      ┆ 18   ┆ … ┆ 9.90566   ┆ 4.360849 ┆ 0.736178 ┆ 보통이다           │
│ 4    ┆ 81   ┆ 26.666667 ┆ 8    ┆ … ┆ 38.915094 ┆ 4.360849 ┆ 0.736178 ┆ 중요하다           │
│ 5    ┆ 67   ┆ 33.333333 ┆ 65   ┆ … ┆ 49.528302 ┆ 4.360849 ┆ 0.736178 ┆ 매우 중요하다      │
└──────┴──────┴───────────┴──────┴───┴───────────┴──────────┴──────────┴───

C:\Users\hyeon\AppData\Local\Temp\ipykernel_13024\3325326262.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select("응답", "2022", "2023", "2024").melt(id_vars=["응답"], variable_name="Year", value_name="Count")


alt.Chart(...)

In [132]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
title = "연구윤리 관련 규정 및 지침의 중요도"
Q ="Q3_4"

# 1) counting
_2022 = pl.DataFrame({
    Q: importance.keys(),
    "2022": [1, 2, 27, 90, 43],
})

_2023 = pl.DataFrame({
    Q: importance.keys(),
    "2023": [0, 1, 24, 86, 59],
})

data = get(Q, _2022, _2023, importance)
draw_graph(data, title, importance)

shape: (5, 14)
┌──────┬──────┬───────────┬──────┬───┬────────┬─────────┬──────────┬────────────────────┐
│ Q3_4 ┆ 2022 ┆ pct2022   ┆ 2023 ┆ … ┆ pctAll ┆ meanAll ┆ stdAll   ┆ 응답               │
│ ---  ┆ ---  ┆ ---       ┆ ---  ┆   ┆ ---    ┆ ---     ┆ ---      ┆ ---                │
│ i64  ┆ i64  ┆ f64       ┆ i64  ┆   ┆ f64    ┆ f64     ┆ f64      ┆ str                │
╞══════╪══════╪═══════════╪══════╪═══╪════════╪═════════╪══════════╪════════════════════╡
│ 1    ┆ 1    ┆ 6.666667  ┆ 0    ┆ … ┆ 0.2    ┆ 4.178   ┆ 0.725476 ┆ 전혀 중요하지 않다 │
│ 2    ┆ 2    ┆ 13.333333 ┆ 1    ┆ … ┆ 1.6    ┆ 4.178   ┆ 0.725476 ┆ 중요하지 않다      │
│ 3    ┆ 27   ┆ 20.0      ┆ 24   ┆ … ┆ 13.0   ┆ 4.178   ┆ 0.725476 ┆ 보통이다           │
│ 4    ┆ 90   ┆ 26.666667 ┆ 86   ┆ … ┆ 50.6   ┆ 4.178   ┆ 0.725476 ┆ 중요하다           │
│ 5    ┆ 43   ┆ 33.333333 ┆ 59   ┆ … ┆ 34.6   ┆ 4.178   ┆ 0.725476 ┆ 매우 중요하다      │
└──────┴──────┴───────────┴──────┴───┴────────┴─────────┴──────────┴────────────────────┘


C:\Users\hyeon\AppData\Local\Temp\ipykernel_13024\3325326262.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select("응답", "2022", "2023", "2024").melt(id_vars=["응답"], variable_name="Year", value_name="Count")


alt.Chart(...)

In [133]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
title = "국내·외 연구부정행위에 대한 검증 및 제재 중요도"
Q ="Q3_5"

# 1) counting
_2022 = pl.DataFrame({
    Q: importance.keys(),
    "2022": [1, 1, 28, 71, 62],
})

_2023 = pl.DataFrame({
    Q: importance.keys(),
    "2023": [0, 2, 24, 77, 67],
})

data = get(Q, _2022, _2023, importance)
draw_graph(data, title, importance)

shape: (5, 14)
┌──────┬──────┬───────────┬──────┬───┬────────┬─────────┬──────────┬────────────────────┐
│ Q3_5 ┆ 2022 ┆ pct2022   ┆ 2023 ┆ … ┆ pctAll ┆ meanAll ┆ stdAll   ┆ 응답               │
│ ---  ┆ ---  ┆ ---       ┆ ---  ┆   ┆ ---    ┆ ---     ┆ ---      ┆ ---                │
│ i64  ┆ i64  ┆ f64       ┆ i64  ┆   ┆ f64    ┆ f64     ┆ f64      ┆ str                │
╞══════╪══════╪═══════════╪══════╪═══╪════════╪═════════╪══════════╪════════════════════╡
│ 1    ┆ 1    ┆ 6.666667  ┆ 0    ┆ … ┆ 0.2    ┆ 4.222   ┆ 0.713243 ┆ 전혀 중요하지 않다 │
│ 2    ┆ 1    ┆ 13.333333 ┆ 2    ┆ … ┆ 0.8    ┆ 4.222   ┆ 0.713243 ┆ 중요하지 않다      │
│ 3    ┆ 28   ┆ 20.0      ┆ 24   ┆ … ┆ 13.2   ┆ 4.222   ┆ 0.713243 ┆ 보통이다           │
│ 4    ┆ 71   ┆ 26.666667 ┆ 77   ┆ … ┆ 48.2   ┆ 4.222   ┆ 0.713243 ┆ 중요하다           │
│ 5    ┆ 62   ┆ 33.333333 ┆ 67   ┆ … ┆ 37.6   ┆ 4.222   ┆ 0.713243 ┆ 매우 중요하다      │
└──────┴──────┴───────────┴──────┴───┴────────┴─────────┴──────────┴────────────────────┘


C:\Users\hyeon\AppData\Local\Temp\ipykernel_13024\3325326262.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select("응답", "2022", "2023", "2024").melt(id_vars=["응답"], variable_name="Year", value_name="Count")


alt.Chart(...)

In [134]:
# 
# 라. KIOST에 대한 인식
# 
df = pl.read_csv("data/4_KIOST_research_ethics_system.csv")

title = "연구부정행위 제보 방법 인지 여부"

Q ="Q26"
yesorno = {
    1: "예", 
    2: "아니오"
}

# 1) counting
_2022 = pl.DataFrame({
    Q: yesorno.keys(),
    "2022": [74, 89],
})

_2023 = pl.DataFrame({
    Q: yesorno.keys(),
    "2023": [80, 90],
})

data = get(Q, _2022, _2023, yesorno)
draw_graph(data, title, yesorno)

shape: (2, 14)
┌─────┬──────┬───────────┬──────┬───┬────────┬─────────┬──────────┬────────┐
│ Q26 ┆ 2022 ┆ pct2022   ┆ 2023 ┆ … ┆ pctAll ┆ meanAll ┆ stdAll   ┆ 응답   │
│ --- ┆ ---  ┆ ---       ┆ ---  ┆   ┆ ---    ┆ ---     ┆ ---      ┆ ---    │
│ i64 ┆ i64  ┆ f64       ┆ i64  ┆   ┆ f64    ┆ f64     ┆ f64      ┆ str    │
╞═════╪══════╪═══════════╪══════╪═══╪════════╪═════════╪══════════╪════════╡
│ 1   ┆ 74   ┆ 33.333333 ┆ 80   ┆ … ┆ 48.2   ┆ 1.518   ┆ 0.499676 ┆ 예     │
│ 2   ┆ 89   ┆ 66.666667 ┆ 90   ┆ … ┆ 51.8   ┆ 1.518   ┆ 0.499676 ┆ 아니오 │
└─────┴──────┴───────────┴──────┴───┴────────┴─────────┴──────────┴────────┘


C:\Users\hyeon\AppData\Local\Temp\ipykernel_13024\3325326262.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select("응답", "2022", "2023", "2024").melt(id_vars=["응답"], variable_name="Year", value_name="Count")


alt.Chart(...)

In [135]:
# 
# 라. KIOST에 대한 인식
# 
# 2) KIOST의 연구부정행위 제보 접수 및 처리 절차 인지
title = "연구부정행위 제보 접수 및 처리의 적정성"
Q ="Q28"

# 1) counting
_2022 = pl.DataFrame({
    Q: yesorno_5.keys(),
    "2022": [4, 14, 88, 44, 13],
})

_2023 = pl.DataFrame({
    Q: yesorno_5.keys(),
    "2023": [7, 19, 78, 51, 15],
})

data = get(Q, _2022, _2023, yesorno_5)
draw_graph(data, title, yesorno_5)


shape: (5, 14)
┌─────┬──────┬───────────┬──────┬───┬────────┬─────────┬──────────┬──────────────────┐
│ Q28 ┆ 2022 ┆ pct2022   ┆ 2023 ┆ … ┆ pctAll ┆ meanAll ┆ stdAll   ┆ 응답             │
│ --- ┆ ---  ┆ ---       ┆ ---  ┆   ┆ ---    ┆ ---     ┆ ---      ┆ ---              │
│ i64 ┆ i64  ┆ f64       ┆ i64  ┆   ┆ f64    ┆ f64     ┆ f64      ┆ str              │
╞═════╪══════╪═══════════╪══════╪═══╪════════╪═════════╪══════════╪══════════════════╡
│ 1   ┆ 4    ┆ 6.666667  ┆ 7    ┆ … ┆ 2.2    ┆ 3.374   ┆ 0.879843 ┆ 전혀 그렇지 않다 │
│ 2   ┆ 14   ┆ 13.333333 ┆ 19   ┆ … ┆ 9.6    ┆ 3.374   ┆ 0.879843 ┆ 그렇지 않다      │
│ 3   ┆ 88   ┆ 20.0      ┆ 78   ┆ … ┆ 47.6   ┆ 3.374   ┆ 0.879843 ┆ 보통이다         │
│ 4   ┆ 44   ┆ 26.666667 ┆ 51   ┆ … ┆ 29.8   ┆ 3.374   ┆ 0.879843 ┆ 그렇다           │
│ 5   ┆ 13   ┆ 33.333333 ┆ 15   ┆ … ┆ 10.8   ┆ 3.374   ┆ 0.879843 ┆ 매우 그렇다      │
└─────┴──────┴───────────┴──────┴───┴────────┴─────────┴──────────┴──────────────────┘


C:\Users\hyeon\AppData\Local\Temp\ipykernel_13024\3325326262.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select("응답", "2022", "2023", "2024").melt(id_vars=["응답"], variable_name="Year", value_name="Count")


alt.Chart(...)

In [136]:
# 
# 라. KIOST에 대한 인식
# 
# 3) KIOST의 연구부정행위 검증이 공정한가에 대한 평가

title = "연구부정행위 검증의 공정성"
Q ="Q29"

# 1) counting
_2022 = pl.DataFrame({
    Q: yesorno_5.keys(),
    "2022": [4, 14, 84, 47, 14],
})

_2023 = pl.DataFrame({
    Q: yesorno_5.keys(),
    "2023": [1, 14, 74, 68, 13],
})

data = get(Q, _2022, _2023, yesorno_5)
draw_graph(data, title, yesorno_5)

ColumnNotFoundError: 연구부정행위 검증의 공정성

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'group_by' <---
DF ["ID", "Q21_1", "Q21_2", "Q21_3"]; PROJECT */44 COLUMNS; SELECTION: None

In [148]:
# 
# 마. KIOST 연구자의 연구윤리 지수 수준
# 

research_ethics_score = pl.read_csv("data/5_1_research_ethics_score.csv")

# *Q39_1: 동물실험에 대한 연구윤리
#
공정 = (pl.col("Q38_1") + pl.col("Q38_2") + pl.col("Q38_3") + pl.col("Q38_4")) / 4
정직 = (pl.col("Q38_5") + pl.col("Q38_6") + pl.col("Q38_7") + pl.col("Q38_8")) / 4
존중 = pl.when(pl.col("Q39_1").is_null()).then(
    (pl.col("Q38_9") + pl.col("Q38_10") + pl.col("Q38_11")) / 3
).otherwise(
    (pl.col("Q38_9") + pl.col("Q38_10") + pl.col("Q38_11") + pl.col("Q39_1")) / 4
)

책임 = (pl.col("Q38_12") + pl.col("Q38_13") + pl.col("Q38_14") + pl.col("Q38_15")) / 4
투명성 = (pl.col("Q38_16") + pl.col("Q38_17") + pl.col("Q38_18") + pl.col("Q38_19")) / 4
all = (공정 + 정직 + 존중 + 책임 + 투명성) / 5

by_area = research_ethics_score.select([
    pl.col("ID"),
    공정.alias("공정"),
    정직.alias("정직"),
    존중.alias("존중"),
    책임.alias("책임"),
    투명성.alias("투명성"),
    all.alias("전체")
])

by_area.mean().select(
    pl.col("공정").round(2),
    pl.col("정직").round(2),
    pl.col("존중").round(2),
    pl.col("책임").round(2),
    pl.col("투명성").round(2),
    pl.col("전체").round(2)
)

공정,정직,존중,책임,투명성,전체
f64,f64,f64,f64,f64,f64
4.23,4.47,4.42,4.23,4.3,4.33
